In [1]:
import wandb
import torch
import torchvision
import os
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torch.optim import Adam

In [2]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
wandb.login(key='3e2fb4c6e149ab3e746227ccd153039708a6d55a')
PROJECT_NAME = 'Assignment2'
ENTITY = 'cs23m004-indian-institute-of-technology-madras'

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs23m004 (cs23m004-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
# Define the data directory
DATA_DIR = '/kaggle/input/inaturalist12k/inaturalist_12K'
PATH_TRAIN_VAL = os.path.join(DATA_DIR, 'train')
PATH_TEST = os.path.join(DATA_DIR, 'test')

In [5]:
config = {
    "method": "bayes",
    "name": "fix",
    "project": "Assignment2",
    "entity": "cs23m004-indian-institute-of-technology-madras",
    "metric": {
        "name": "valid_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "num_filters": {
            "values": [
                [32, 64, 128, 256, 512]
            ]
        },
        "conv_filter_size": {
            "values": [
                [7, 7, 3, 3, 3],
                [3, 3, 3, 3, 3],
                [3, 3, 3, 7, 7]
            ]
        },
        "batch_size": {
            "values": [16, 32, 64]
        },
        "conv_activation": {
            "values": [
                ["relu", "relu", "relu", "relu", "relu"]
            ]
        },
        "dense_activation": {
            "values": ["relu", "sigmoid"]
        },
        "pool_filter_size": {
            "values": [
                [2, 2, 2, 2, 2]
            ]
        },
        "dense_layer": {
            "values": [32, 64, 128, 512]
        },
        "dropout": {
            "values": [0.0, 0.4, 0.5, 0.6]
        },
        "learning_rate": {
            "values": [0.001, 0.0001]
        },
        "num_epochs": {
            "values": [10, 15, 20, 25]
        },
        "data_augmentation": {
            "values": [True]
        },
        "batch_norm": {
            "values": [True, False]
        }
    }
}


In [6]:
def get_data(data_path=PATH_TRAIN_VAL, batch_size=32, augmentation=False):
    '''
    Loads and splits the dataset from the specified directory into training and validation sets.

    By default, it performs an 80-20 random split, with 20% of the data used for validation.
    If the `augmentation` flag is set to True, additional data augmentation techniques
    are applied to the training dataset.

    The function returns two DataLoader objects: one for the training set and one for the validation set.

    Parameters:
        data_path (str): Path to the directory containing the dataset.
        augmentation (bool): Whether to apply data augmentation to the training data.

    Returns:
        train_loader, val_loader: DataLoader instances for training and validation.
'''

   # Set up preprocessing and augmentation pipelines for training and validation
    data_transforms = {
        "val": transforms.Compose([
            transforms.Resize((256, 256)),                        # Apply same resizing for validation
            transforms.ToTensor(),                                # Convert to tensor format
            transforms.Normalize(                                 # Apply normalization
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ]),
        "train": transforms.Compose([
            transforms.Resize((256, 256)),                        # Resize all training images
            transforms.RandomHorizontalFlip(),                    # Randomly flip images horizontally
            transforms.ToTensor(),                                # Convert images to PyTorch tensors
            transforms.Normalize(                                 # Normalize using ImageNet stats
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
        
    }

    totalsize = 0
    aug = augmentation
    if aug :
        print("applying Augmentations")
    if aug:
        # Apply augmentations
        extra_transforms = [
            transforms.RandomVerticalFlip(),                      # Random vertical flip
            transforms.RandomRotation(20),                        # Rotate images randomly up to 20 degrees
            transforms.ColorJitter(                               
                brightness=0.1, contrast=0.1,                     
                saturation=0.1, hue=0.1                           # Slight color distortion
            )
        ]
        data_transforms['train'].transforms.insert(2, extra_transforms[0])  # Vertical flip
        data_transforms['train'].transforms.insert(3, extra_transforms[1])  # Rotation
        data_transforms['train'].transforms.insert(4, extra_transforms[2])  # Color jitter

    
    # Load the datasets using ImageFolder
    image_datasets = {
        "train": datasets.ImageFolder(data_path, data_transforms["train"]),
        "val": datasets.ImageFolder(data_path, data_transforms["val"])
    }
    
    # Perform an 80-20 split on the training dataset
    splitratio = 0.8
    ttlsize = len(image_datasets["train"])
    train_size = int(splitratio * ttlsize)
    val_size = len(image_datasets["train"]) - train_size
    print("")
    train_dataset, val_dataset = random_split(image_datasets["train"], [train_size, val_size])
    
    # Creating dataloaders for train and validation sets
    print("")
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    return train_loader, val_loader

In [7]:
class CNN(nn.Module):
    
    def __init__(self, img_shape, conv_activation, dense_activation, num_filters, conv_filter_size, pool_filter_size, batch_norm, dense_layer, dropout):
        
        super(CNN, self).__init__()
        print("")
        # Save configuration parameters
        # Store initialization parameters
        self.batch_norm = batch_norm
        self.img_shape = img_shape
        self.conv_activation = conv_activation
        print("")
        self.num_filters = num_filters
        self.pool_filter_size = pool_filter_size
        self.dense_activation = dense_activation
        print("")
        self.conv_filter_size = conv_filter_size
        self.dropout = dropout
        self.dense_layer = dense_layer
        
        # Initialize the convolutional layer sequence
        self.conv_layers = nn.Sequential()
        print("")
        # Extract the number of channels from the input shape
        input_channels = img_shape[0]


       # Loop through each convolutional layer configuration
        for i in range(5):
            # Add convolutional layer
            self.conv_layers.add_module(f'conv{i+1}',nn.Conv2d(in_channels=input_channels,out_channels=num_filters[i],kernel_size=conv_filter_size[i]))
        
            # Optionally include batch normalization
            if batch_norm: print("applying batch normalisation")
            if batch_norm:
                self.conv_layers.add_module(f'bn{i+1}', nn.BatchNorm2d(num_filters[i]))
        
            # Select activation function based on config
            if conv_activation[i] == 'sigmoid':
                 activation_fn = nn.Sigmoid()
            else :
                 activation_fn = nn.ReLU()
            
            self.conv_layers.add_module(f'activation{i+1}', activation_fn)
        
            # Apply max pooling layer
            self.conv_layers.add_module(f'pool{i+1}',nn.MaxPool2d(kernel_size=pool_filter_size[i],stride=2))
        
            # Update input channel count for the next conv layer
            input_channels = num_filters[i]

        # Compute the flattened size after conv layers by forwarding a dummy input
        dummy_input = torch.randn(1, *img_shape)
        with torch.no_grad():
            dummy_output = self.conv_layers(dummy_input)
        self.flatten_dim = dummy_output.view(1, -1).size(1)

        # Fully connected layers
        self.fc_layers = nn.Sequential(nn.Linear(self.flatten_dim, dense_layer),nn.Sigmoid() if dense_activation == 'sigmoid' else nn.ReLU(),nn.Dropout(dropout),
            nn.Linear(dense_layer, 10)  # 10 output classes
        )

    
    
  
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc_layers(x)
        return x

In [8]:
def train():
    # Initialize a new Weights & Biases run
    with wandb.init(project=PROJECT_NAME, entity=ENTITY):
        print("Wandb initiates")
        config = wandb.config  # Access hyperparameter config from wandb

        # Load training and validation data using specified batch size and augmentation flag
        train_loader, val_loader = get_data(augmentation=config.data_augmentation,batch_size=config.batch_size)
        # Create a unique name for the run based on configuration settings
        run_name = (
            f"batch_size={config.batch_size}_"
            f"batch_norm={config.batch_norm}_"
            f"num_filters={config.num_filters}_"
            f"conv_activation={config.conv_activation}_"
            f"dense_activation={config.dense_activation}_"
            f"conv_filter_size={config.conv_filter_size}_"
            f"pool_filter_size={config.pool_filter_size}_"
            f"dense_layer={config.dense_layer}_"
            f"dropout={config.dropout}_"
            f"learning_rate={config.learning_rate}_"
            f"num_epochs={config.num_epochs}_"
            f"data_augmentation={config.data_augmentation}"
        )
        wandb.run.name = run_name  # Assign the generated name to the run

        # CNN model using the provided config
        model = CNN(
            conv_activation=config.conv_activation,
            num_filters=config.num_filters,
            dense_activation=config.dense_activation,
            img_shape=(3, 256, 256),
            conv_filter_size=config.conv_filter_size,
            pool_filter_size=config.pool_filter_size,
            batch_norm=config.batch_norm,
            dense_layer=config.dense_layer,
            dropout=config.dropout
        ).to(device)
        print("Model initialised")

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = Adam(model.parameters(), lr=config.learning_rate)

        # Training process begins
        for epoch in range(config.num_epochs):
            print(f"epoch : {epoch + 1}")
            model.train()
            train_loss, correct_preds, total_samples = 0.0, 0, 0

            for batch_idx, (inputs, labels) in enumerate(train_loader):
                if batch_idx % 20 == 0:
                    print(f"batch {batch_idx}")
                    
                # Move data to appropriate device (CPU or GPU)
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass and backpropagation
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # Log batch loss to wandb
                wandb.log({"batch_loss": loss.item()})

                # Update training metrics
                train_loss = train_loss+loss.item()
                _, predictions = torch.max(outputs, 1)
                total_samples = total_samples+labels.size(0)
                correct_preds += (predictions == labels).sum().item()

            # Calculate average training loss and accuracy
            lenloader = len(train_loader)
            avg_train_loss = train_loss / lenloader
            train_accuracy = correct_preds / total_samples

            # Validation phase
            model.eval()
            val_loss, val_correct, val_total = 0.0, 0, 0

            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    val_loss = val_loss + loss.item()
                    _, predictions = torch.max(outputs, 1)
                    val_total = val_total + labels.size(0)
                    correctpred = (predictions == labels).sum().item()
                    val_correct = val_correct + correctpred 

            # Compute validation metrics
            val_len = len(val_loader)
            avg_val_loss = val_loss / val_len
            val_accuracy = val_correct / val_total

            # Log metrics to wandb for this epoch
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": avg_train_loss,
                "train_accuracy": train_accuracy,
                "valid_loss": avg_val_loss,
                "valid_accuracy": val_accuracy
            })

            # Print epoch summary
            print(
                f"Epoch [{epoch + 1}/{config.num_epochs}] "
                f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
                f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}"
            )


In [ ]:
# Initialize and run the sweep
sweep_id = wandb.sweep(config)
wandb.agent(sweep_id, function=train, count = 1)


Create sweep with ID: 1u7mw8kg
Sweep URL: https://wandb.ai/cs23m004-indian-institute-of-technology-madras/Assignment2/sweeps/1u7mw8kg


wandb: Agent Starting Run: bb1eq2kj with config:
wandb: 	batch_norm: True
wandb: 	batch_size: 16
wandb: 	conv_activation: ['relu', 'relu', 'relu', 'relu', 'relu']
wandb: 	conv_filter_size: [3, 3, 3, 7, 7]
wandb: 	data_augmentation: True
wandb: 	dense_activation: relu
wandb: 	dense_layer: 32
wandb: 	dropout: 0
wandb: 	learning_rate: 0.0001
wandb: 	num_epochs: 25
wandb: 	num_filters: [32, 64, 128, 256, 512]
wandb: 	pool_filter_size: [2, 2, 2, 2, 2]


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Wandb initiates
applying Augmentations






applying batch normalisation
applying batch normalisation
applying batch normalisation
applying batch normalisation
applying batch normalisation
Model initialised
epoch : 1
batch 0
batch 20
batch 40
